In [8]:
library(Seurat)
library(tidyverse)
library(here)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [10]:
files <- list.files(here("data/tnl"), full.names = TRUE, recursive = TRUE, pattern = 'filtered_feature_bc_matrix', ignore.case = TRUE)
names(files) <- str_extract(files, "(?<=/tnl/)[^/]+")

In [18]:
objs <- list()
for (i in 1:length(files)){
    mat <- Read10X_h5(files[[i]])
    obj <- CreateSeuratObject(counts = mat, min.features = 500, min.cells = 5, project = names(files[i]))
    objs <- append(objs, list(obj))
}

In [16]:
mat <- Read10X_h5(files[[1]])

In [17]:
str(mat)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:3531300] 310 581 744 904 1113 1115 1487 1513 1553 1584 ...
  ..@ p       : int [1:6333] 0 247 535 934 1840 2543 2842 3466 3735 4187 ...
  ..@ Dim     : int [1:2] 38606 6332
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:38606] "DDX11L2" "MIR1302-2HG" "FAM138A" "ENSG00000290826" ...
  .. ..$ : chr [1:6332] "AAACCCAGTATGGAAT-1" "AAACCCAGTCCTCAGG-1" "AAACCCAGTGGTCTAT-1" "AAACCCAGTTCTTGCC-1" ...
  ..@ x       : num [1:3531300] 1 1 1 1 1 1 2 1 1 2 ...
  ..@ factors : list()
